In [37]:
import pandas as pd
import numpy as np
import importlib
import seaborn as sns
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix, roc_auc_score,average_precision_score,f1_score,plot_confusion_matrix
from xgboost import XGBClassifier
import shap
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import plot_roc_curve
from statistics import mean
from sklearn.ensemble import RandomForestClassifier
import re
import pickle
from matplotlib import pyplot
import time
import scispacy
import spacy
import nltk
from tqdm import tqdm

# Functions to generate features

In [2]:
from nltk.corpus import stopwords
import string
#nltk.download('punkt')
#nltk.download('stopwords')

def feature_gen(dataframe,include_bigrams='FALSE'):
    
    meas_method_column_vals = []
    entity_nlp = spacy.load('en_core_sci_sm')
    chemical_disease_nlp = spacy.load('en_ner_bc5cdr_md')
    chem_ent_ratios, seen_chem, chem_count = [], [], []
    human_bigram = []    
    for index, row in dataframe.iterrows():
        abstract = row['abstract']
    #   ------------ Chemical Names ----------------
        entity_doc = entity_nlp(abstract)
        chemical_disease_doc = chemical_disease_nlp(abstract)
        chemical_ents = [ent.text for ent in chemical_disease_doc.ents if ent.label_ == 'CHEMICAL']
        if len(entity_doc.ents) == 0:
            chem_ent_ratios.append(0)
        else:
            chem_ent_ratios.append(len(chemical_ents) / len(entity_doc.ents))
        seen_chem.append(list(set(chemical_ents)))
        chem_count.append(len(chemical_ents))
    #   ------------ Bigram Score --------------
        if include_bigrams == 'TRUE':
            tokens = nltk.word_tokenize(abstract)
            bigrams = nltk.bigrams(tokens)
            stopset = set(stopwords.words('english') + list(string.punctuation))
            milk_bigrams = [(w1, w2) for w1, w2 in bigrams if 
                            (w1.lower() == 'milk' or w2.lower() == 'milk')
                             and (w1.lower() not in stopset and w2.lower() not in stopset)]
            human_bigrams = [(w1, w2) for w1, w2 in milk_bigrams if 
                             (w1.lower() == 'human' or w2.lower() == 'human')
                             and (w1.lower() not in stopset and w2.lower() not in stopset)]
            human_bigram.append(len(human_bigrams) / len(milk_bigrams) if len(milk_bigrams) != 0 else 0)
        else:
            human_bigram.append(0.0)

    dataframe['chem_ent_ratio'] = chem_ent_ratios
    dataframe['chemicals'] = seen_chem
    dataframe['bigram_score'] = human_bigram
    dataframe['chem_term_count'] = chem_count
    
    return dataframe

In [2]:
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 150

def clean_plot(leg=True, grid=None, font=None):
    ax = plt.gca()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    
    axis_color = 'lightgrey'
    ax.spines['bottom'].set_color(axis_color)
    ax.spines['left'].set_color(axis_color)
    ax.tick_params(axis='both', color=axis_color)
    
    if leg:
        ax.legend(frameon = False, loc='upper left', bbox_to_anchor=(1, 1))
        
    if grid is not None:
        plt.grid(color='lightgrey', axis = grid, linestyle='-', linewidth=.5)
        
    if font is not None:
        for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
            ax.get_xticklabels() + ax.get_yticklabels()):
            
            item.set_fontfamily(font['family'])
            item.set_color(font['color'])
            

In [4]:
from src.filter import Filter

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ayan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Loading datasets and generating features

In [10]:
## Garlic and Cocoa

gtrain = pd.read_csv("data/garlic_scoring.csv", encoding='latin1')[['PMID', 'abstract', 'paper', 'mesh_terms', 'qual_terms', 'is_useful']]
gtrain['food'] = 'garlic'
ctrain = pd.read_csv("data/cocoa_scoring.csv", encoding='latin1')[['PMID', 'abstract', 'paper', 'mesh_terms', 'qual_terms', 'is_useful']]
ctrain['food'] = 'cocoa'
gtrain['is_useful'] = gtrain['is_useful'].replace(2, 1, regex=True)
ctrain['is_useful'] = ctrain['is_useful'].replace(2, 1, regex=True)
gtrain = gtrain[gtrain['is_useful'].notnull()]
ctrain = ctrain[ctrain['is_useful'].notnull()]

## Basil

btrain = pd.read_excel("data/basil_scoring.xls", encoding='latin1')[['PMID', 'abstract', 'paper', 'mesh_terms', 'qual_terms', 'is_useful']]
btrain['food'] = 'apple'

for i in range(len(btrain)):
    if btrain['is_useful'].loc[i] == 'x':
        btrain['is_useful'].loc[i] = 0
        
btrain['is_useful'] = btrain['is_useful'].replace(2, 1, regex=True)
btrain = btrain[btrain['is_useful'].notnull()]

## Apple

atrain = pd.read_excel("data/apple_scoring.xls", encoding='latin1')[['PMID', 'abstract', 'paper', 'mesh_terms', 'qual_terms', 'is_useful']]
atrain['food'] = 'apple'

for i in range(len(atrain)):
    if atrain['is_useful'].loc[i] == 'x':
        atrain['is_useful'].loc[i] = 0

atrain['is_useful'] = atrain['is_useful'].replace(2, 1, regex=True)
atrain = atrain[atrain['is_useful'].notnull()]
atrain = atrain[atrain['abstract'].notnull()]
atrain = atrain[atrain['PMID'].notnull()]

## Human Milk database

mtrain_new = pd.read_csv("mBase_15Aug_abstract[chemical_gen].csv")
mtrain_new['food'] = 'milk'
mtrain_new = mtrain_new[mtrain_new['abstract'].notnull()]
mtrain_new = mtrain_new[mtrain_new['PMID'].notnull()]


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
print('Length of the databases for garlic, cocoa, basil, apple, milk: ', len(gtrain), len(ctrain), len(btrain), len(atrain), len(mtrain_new))

Length of the databases for garlic, cocoa, basil, apple, milk:  299 324 93 1653 229


In [7]:
print('Useful and non-useful: ',len(gtrain[gtrain['is_useful'] == 1.0]),len(gtrain[gtrain['is_useful'] == 0.0]))

Useful and non-useful:  77 222


In [8]:
print('Useful and non-useful: ',len(ctrain[ctrain['is_useful'] == 1.0]),len(ctrain[ctrain['is_useful'] == 0.0]))

Useful and non-useful:  93 231


In [9]:
print('Useful and non-useful: ',len(atrain[atrain['is_useful'] == 1.0]),len(atrain[atrain['is_useful'] == 0.0]))

Useful and non-useful:  462 1191


In [309]:
print('Useful and non-useful: ',len(btrain[btrain['is_useful'] == 1.0]),len(btrain[btrain['is_useful'] == 0.0]))

Useful and non-useful:  57 36


In [257]:
def build_all_features(ftrain,include_bigrams='FALSE'):
    fmodel_data = Filter()

    fmodel_data.build_features(input_data = ftrain,is_traindata = True)

    ftrain = feature_gen(ftrain,include_bigrams)

    fmodel_data.data['chem_ent_ratio'] = ftrain['chem_ent_ratio'].values
    fmodel_data.data['chem_term_count'] = ftrain['chem_term_count'].values
    fmodel_data.data['bigram_score'] = ftrain['bigram_score'].values
    
    return fmodel_data

In [260]:
print('----Starting feature generation----')
gdata = build_all_features(gtrain,include_bigrams='FALSE')
print('----DONE----')
cdata = build_all_features(ctrain,include_bigrams='FALSE')
print('----DONE----')
bdata = build_all_features(btrain,include_bigrams='FALSE')
print('----DONE----')

----Starting feature generation----




0it [00:00, ?it/s]



Creating features...


1it [00:00,  4.79it/s]

2it [00:00,  3.97it/s]

3it [00:00,  4.32it/s]

4it [00:00,  4.50it/s]

5it [00:01,  4.80it/s]

6it [00:01,  4.61it/s]

7it [00:01,  5.04it/s]

8it [00:01,  5.15it/s]

9it [00:01,  4.64it/s]

10it [00:02,  4.01it/s]

11it [00:02,  4.18it/s]

12it [00:02,  3.83it/s]

13it [00:03,  4.01it/s]

14it [00:03,  3.81it/s]

15it [00:03,  4.35it/s]

16it [00:03,  4.54it/s]

17it [00:03,  4.16it/s]

18it [00:04,  5.01it/s]

20it [00:04,  5.12it/s]

21it [00:04,  5.56it/s]

22it [00:04,  4.23it/s]

23it [00:05,  4.61it/s]

24it [00:05,  5.35it/s]

25it [00:05,  4.15it/s]

26it [00:05,  3.91it/s]

27it [00:06,  3.51it/s]

28it [00:06,  3.19it/s]

29it [00:06,  3.77it/s]

30it [00:07,  3.96it/s]

31it [00:07,  3.67it/s]

32it [00:07,  3.82it/s]

33it [00:07,  4.07it/s]

34it [00:08,  4.04it/s]

35it [00:08,  3.55it/s]

36it [00:08,  3.72it/s]

37it [00:09,  3.34it/s]

38it [00:09,  4.12it/s]

39it [00:09,  4.46it/s]

40it [00:09,  4.48it/s]

41it [00:09,  4.60it/s]

42it [00:

----DONE----


...

----DONE----




0it [00:00, ?it/s]

Creating features...




1it [00:00,  2.91it/s]

2it [00:00,  2.69it/s]

3it [00:01,  2.97it/s]

4it [00:01,  2.85it/s]

5it [00:01,  2.80it/s]

6it [00:02,  2.93it/s]

7it [00:02,  2.79it/s]

8it [00:02,  2.80it/s]

9it [00:03,  2.72it/s]

10it [00:03,  2.88it/s]

11it [00:04,  2.45it/s]

12it [00:04,  2.51it/s]

13it [00:04,  2.47it/s]

14it [00:05,  2.28it/s]

15it [00:05,  2.25it/s]

16it [00:06,  2.42it/s]

17it [00:06,  2.51it/s]

18it [00:06,  2.98it/s]

19it [00:06,  3.71it/s]

20it [00:07,  4.33it/s]

21it [00:07,  3.83it/s]

22it [00:07,  3.90it/s]

23it [00:08,  3.10it/s]

24it [00:08,  2.55it/s]

25it [00:08,  2.65it/s]

26it [00:09,  2.94it/s]

27it [00:09,  3.10it/s]

28it [00:09,  3.67it/s]

29it [00:09,  4.32it/s]

30it [00:10,  3.08it/s]

31it [00:10,  2.53it/s]

32it [00:11,  2.50it/s]

33it [00:11,  2.76it/s]

34it [00:11,  2.77it/s]

35it [00:12,  2.83it/s]

36it [00:12,  2.88it/s]

37it [00:12,  2.91it/s]

38it [00:13,  2.78it/s]

39it [00:13,  2.87it/s]

40it [00:14,  2.84it/s]

41it [0

----DONE----


In [261]:
print('----Starting feature generation----')
adata = build_all_features(atrain,include_bigrams='FALSE')
print('----DONE----')
mdata = build_all_features(mtrain_new,include_bigrams='TRUE')
print('----DONE----')

----Starting feature generation----




0it [00:00, ?it/s]

1it [00:00,  6.15it/s]

Creating features...




2it [00:00,  5.51it/s]

3it [00:00,  5.60it/s]

4it [00:00,  5.31it/s]

5it [00:00,  5.18it/s]

6it [00:01,  5.26it/s]

7it [00:01,  4.76it/s]

8it [00:01,  4.80it/s]

9it [00:01,  4.74it/s]

10it [00:02,  4.95it/s]

11it [00:02,  5.79it/s]

12it [00:02,  5.54it/s]

13it [00:02,  4.37it/s]

14it [00:02,  4.19it/s]

15it [00:03,  3.55it/s]

16it [00:03,  3.39it/s]

18it [00:03,  3.87it/s]

19it [00:04,  4.03it/s]

20it [00:04,  4.35it/s]

21it [00:04,  3.84it/s]

22it [00:04,  4.12it/s]

23it [00:05,  4.36it/s]

24it [00:05,  4.62it/s]

25it [00:05,  5.16it/s]

26it [00:05,  4.40it/s]

27it [00:06,  4.22it/s]

28it [00:06,  4.28it/s]

29it [00:06,  4.09it/s]

30it [00:06,  4.00it/s]

31it [00:07,  3.86it/s]

32it [00:07,  3.43it/s]

33it [00:07,  3.44it/s]

34it [00:07,  3.57it/s]

35it [00:08,  3.77it/s]

36it [00:08,  4.47it/s]

37it [00:08,  4.53it/s]

38it [00:08,  4.19it/s]

39it [00:09,  4.37it/s]

40it [00:09,  3.94it/s]

41it [00:09,  4.07it/s]

42it [00:09,  3.96it/s]

43it [

518it [02:26,  4.19it/s]

519it [02:26,  3.94it/s]

520it [02:26,  3.57it/s]

521it [02:27,  3.76it/s]

522it [02:27,  3.41it/s]

523it [02:27,  3.28it/s]

524it [02:27,  3.52it/s]

525it [02:28,  3.56it/s]

526it [02:28,  3.46it/s]

527it [02:28,  3.81it/s]

528it [02:28,  4.40it/s]

529it [02:29,  4.58it/s]

530it [02:29,  3.96it/s]

531it [02:29,  3.78it/s]

532it [02:30,  3.29it/s]

533it [02:30,  3.12it/s]

534it [02:30,  3.09it/s]

535it [02:31,  3.06it/s]

536it [02:31,  3.29it/s]

537it [02:31,  3.17it/s]

538it [02:31,  3.25it/s]

539it [02:32,  3.53it/s]

540it [02:32,  3.74it/s]

541it [02:32,  3.64it/s]

542it [02:32,  3.71it/s]

543it [02:33,  3.78it/s]

544it [02:33,  3.39it/s]

545it [02:33,  3.60it/s]

546it [02:34,  3.34it/s]

547it [02:34,  3.67it/s]

548it [02:34,  3.30it/s]

549it [02:35,  3.04it/s]

550it [02:35,  2.91it/s]

551it [02:35,  3.08it/s]

552it [02:36,  3.09it/s]

553it [02:36,  3.07it/s]

554it [02:36,  3.40it/s]

555it [02:37,  3.06it/s]

556it [02:37

...

----DONE----




0it [00:00, ?it/s]

Creating features...




1it [00:00,  2.58it/s]

2it [00:00,  3.11it/s]

3it [00:00,  3.69it/s]

4it [00:00,  3.68it/s]

5it [00:01,  3.57it/s]

6it [00:01,  3.32it/s]

7it [00:01,  3.70it/s]

9it [00:02,  4.41it/s]

10it [00:02,  4.72it/s]

11it [00:02,  4.52it/s]

12it [00:02,  4.52it/s]

14it [00:03,  4.46it/s]

15it [00:03,  4.37it/s]

16it [00:03,  3.37it/s]

17it [00:04,  3.92it/s]

18it [00:04,  3.61it/s]

19it [00:04,  3.32it/s]

20it [00:04,  3.94it/s]

21it [00:05,  4.55it/s]

22it [00:05,  4.37it/s]

23it [00:05,  4.23it/s]

24it [00:05,  3.94it/s]

25it [00:06,  3.54it/s]

26it [00:06,  3.72it/s]

27it [00:06,  3.44it/s]

28it [00:06,  3.65it/s]

29it [00:07,  3.55it/s]

30it [00:07,  3.84it/s]

31it [00:07,  3.95it/s]

32it [00:07,  4.44it/s]

33it [00:08,  3.63it/s]

34it [00:08,  3.19it/s]

35it [00:08,  3.23it/s]

36it [00:09,  3.63it/s]

37it [00:09,  3.48it/s]

38it [00:09,  3.81it/s]

39it [00:09,  3.83it/s]

40it [00:10,  3.69it/s]

41it [00:10,  3.17it/s]

42it [00:10,  3.12it/s]

43it [

----DONE----


In [262]:
gdata_features_class = gdata.data.copy()
cdata_features_class = cdata.data.copy()
bdata_features_class = bdata.data.copy()
adata_features_class = adata.data.copy()
mdata_features_class = mdata.data.copy()

In [263]:
gdata_features_class.to_csv('data_with_feature/gdata_features_class.csv')
cdata_features_class.to_csv('data_with_feature/cdata_features_class.csv')
bdata_features_class.to_csv('data_with_feature/bdata_features_class.csv')
adata_features_class.to_csv('data_with_feature/adata_features_class.csv')
mdata_features_class.to_csv('data_with_feature/mdata_features_class.csv')

In [3]:
gdata_features_class = pd.read_csv('data_with_feature/gdata_features_class.csv')
cdata_features_class = pd.read_csv('data_with_feature/cdata_features_class.csv')
bdata_features_class = pd.read_csv('data_with_feature/bdata_features_class.csv')
adata_features_class = pd.read_csv('data_with_feature/adata_features_class.csv')
mdata_features_class = pd.read_csv('data_with_feature/mdata_features_class.csv')

# Doc2vec trained on FoodBase

In [4]:
fmine = pd.read_csv('FoodBase_Abstracts_embeddings.csv')

In [5]:
embedding_list = []

for index, row in fmine.iterrows():
    local_list = []
    for x in row['vectors'].replace('[','').replace(']','').replace('\n',' ').replace('  ',' ').replace('   ',' ').split(' '):
        if x != '':
            local_list.append(float(x))
    embedding_list.append(local_list)
    
fmine['embeddings'] = embedding_list

In [6]:
abstract_embedding_dict = dict()

for index, row in fmine.iterrows():
    abstract_embedding_dict[row['abstract']] = row['embeddings']

In [7]:
print('Dimension of Doc2Vec: ', len(embedding_list[0]))

Dimension of Doc2Vec:  64


In [8]:
fmine_expanded = pd.concat([fmine, fmine['embeddings'].apply(pd.Series)], axis = 1)

In [11]:
gdata_train_doc2vec = pd.merge(gtrain, fmine_expanded[fmine_expanded['food'] == 'garlic'], on=['abstract'])
gdata_features_class_doc2vec = pd.merge(gdata_train_doc2vec, gdata_features_class, on=['PMID'])
gdata_features_class_doc2vec = gdata_features_class_doc2vec.drop(columns=['PMID','abstract','paper','mesh_terms','qual_terms','is_useful_x','food_x','is_useful_y','food_y','vectors','embeddings'])

In [12]:
cdata_train_doc2vec = pd.merge(ctrain, fmine_expanded[fmine_expanded['food'] == 'cocoa'], on=['abstract'])
cdata_features_class_doc2vec = pd.merge(cdata_train_doc2vec, cdata_features_class, on=['PMID'])
cdata_features_class_doc2vec = cdata_features_class_doc2vec.drop(columns=['PMID','abstract','paper','mesh_terms','qual_terms','is_useful_x','food_x','is_useful_y','food_y','vectors','embeddings'])

In [13]:
bdata_train_doc2vec = pd.merge(btrain, fmine_expanded[fmine_expanded['food'] == 'basil'], on=['abstract'])
bdata_features_class_doc2vec = pd.merge(bdata_train_doc2vec, bdata_features_class, on=['PMID'])
bdata_features_class_doc2vec = bdata_features_class_doc2vec.drop(columns=['PMID','abstract','paper','mesh_terms','qual_terms','is_useful_x','food_x','is_useful_y','food_y','vectors','embeddings'])

In [14]:
adata_train_doc2vec = pd.merge(atrain, fmine_expanded[fmine_expanded['food'] == 'apple'], on=['abstract'])
adata_features_class_doc2vec = pd.merge(adata_train_doc2vec, adata_features_class, on=['PMID'])
adata_features_class_doc2vec = adata_features_class_doc2vec.drop(columns=['PMID','abstract','paper','mesh_terms','qual_terms','is_useful_x','food_x','is_useful_y','food_y','vectors','embeddings'])

In [15]:
mdata_train_doc2vec = pd.merge(mtrain_new, fmine_expanded[fmine_expanded['food'] == 'human milk'], on=['abstract'])
mdata_features_class_doc2vec = pd.merge(mdata_train_doc2vec, mdata_features_class, on=['PMID'])
mdata_features_class_doc2vec = mdata_features_class_doc2vec.drop(columns=['PMID','abstract','paper','mesh_terms','qual_terms','is_useful_x','food_x','is_useful_y','food_y','vectors','embeddings'])

In [16]:
mdata_features_class_doc2vec = mdata_features_class_doc2vec.drop(columns=['journal','mesh_UIds','qual_UIds','webpage','year','source','measmethod','chem_ent_ratio_x','chemicals','bigram_score_x','chem_term_count_x'])

In [17]:
mdata_features_class_doc2vec = mdata_features_class_doc2vec.rename(columns={'chem_ent_ratio_y':'chem_ent_ratio','chem_term_count_y':'chem_term_count','bigram_score_y':'bigram_score'})

# Normalize the features

In [281]:
def normalize_features(total_data):
    total_data_features = total_data.drop(columns=['class'])
    for col_feature in list(total_data_features.columns): 
        col_list = total_data_features[col_feature].tolist()
        if col_list != [0.0] * len(col_list):
            col_list_normalized = [(x-np.mean(col_list))/np.std(col_list) for x in col_list]
        else:
            col_list_normalized = col_list
        total_data_features[col_feature] = col_list_normalized
    # total_data_features_normalized = (total_data_features-total_data_features.mean())/total_data_features.std()
    # total_data_features_normalized = (total_data_features-total_data_features.min())/(total_data_features.max()-total_data_features.min())
    total_data_features['class'] = total_data['class'].tolist()
    
    return total_data_features

In [282]:
gdata_features_class_normalized = normalize_features(gdata_features_class)
cdata_features_class_normalized = normalize_features(cdata_features_class)
bdata_features_class_normalized = normalize_features(bdata_features_class)
adata_features_class_normalized = normalize_features(adata_features_class)
mdata_features_class_normalized = normalize_features(mdata_features_class)

# Performances on seen food 

In [21]:
def xgboost_model(x,y,kfold,n_splits=10):

    cross_val_model_fm1 = RandomForestClassifier(max_depth=80,random_state=0)
    cross_val_model_fm2 = RandomForestClassifier(max_depth=80,random_state=0)
    
    kfold = StratifiedKFold(n_splits)
    
    auc_fm1 = []
    aup_fm1 = []
    f1_score_list_fm1 = []
    
    auc_fm2 = []
    aup_fm2 = []
    f1_score_list_fm2 = []
    
    for i,(train,test) in enumerate(kfold.split(x,y)):
        
        x_old = x.drop(columns=['chem_ent_ratio','chem_term_count','bigram_score'])
        
        cross_val_model_fm2.fit(x.loc[train],y.loc[train])
        cross_val_model_fm1.fit(x_old.loc[train],y.loc[train])
        
        #viz_fm1 = plot_roc_curve(cross_val_model_fm1, x_old.loc[test], y.loc[test],
        #                 name='ROC fold {}'.format(i),
        #                 alpha=0.3, lw=1)
        #viz_fm2 = plot_roc_curve(cross_val_model_fm2, x.loc[test], y.loc[test],
        #                 name='ROC fold {}'.format(i),
        #                 alpha=0.3, lw=1)
        
        y_predicted_fm1 = cross_val_model_fm1.predict(x_old.loc[test])
        y_predicted_fm2 = cross_val_model_fm2.predict(x.loc[test])
        
        auc_fm1.append(roc_auc_score(np.array(y.loc[test].tolist()),y_predicted_fm1))
        aup_fm1.append(average_precision_score(np.array(y.loc[test].tolist()),y_predicted_fm1))
        f1_score_list_fm1.append(f1_score(np.array(y.loc[test].tolist()),y_predicted_fm1))
        
        auc_fm2.append(roc_auc_score(np.array(y.loc[test].tolist()),y_predicted_fm2))
        aup_fm2.append(average_precision_score(np.array(y.loc[test].tolist()),y_predicted_fm2))
        f1_score_list_fm2.append(f1_score(np.array(y.loc[test].tolist()),y_predicted_fm2))
        # plot_confusion_matrix(cross_val_model,x.loc[test],y.loc[test])
        
    print('------------------------------------------------------------------------------------------------------')
    #print('Average AUC for FM1 and FM2: ', mean(auc_fm1), mean(auc_fm2))
    #print('Average AUP for FM1 and FM2:', mean(aup_fm1), mean(aup_fm2))
    #print('Average f1 Socre FM1 and FM2:', mean(f1_score_list_fm1), mean(f1_score_list_fm2))
    print('Average and SD of AUC for FM2: ', mean(auc_fm2), np.std(auc_fm2))
    print('Average and SD of AUP for FM2:', mean(aup_fm2), np.std(aup_fm2))
    print('Average and SD of f1 Socre FM2:', mean(f1_score_list_fm2), np.std(f1_score_list_fm2))
    print('------------------------------------------------------------------------------------------------------')

    return mean(auc_fm2), mean(aup_fm2), mean(f1_score_list_fm2)

In [39]:
def get_cross_validation_seen_food(fdata_features_class_normalized):

    repeat = 10

    for _ in tqdm(range(repeat)):
        oversample = SMOTE()
        y = fdata_features_class_normalized.copy()['class']
        X = fdata_features_class_normalized.copy().drop('class', axis = 1)
        X_smote, y_smote = oversample.fit_resample(X, y)
        kfold = StratifiedKFold(n_splits=10)
        auc_fm2, aup_fm2, f1_fm2 = xgboost_model(X_smote, y_smote,kfold,n_splits=10)
        
    return 

In [40]:
## Garlic
get_cross_validation_seen_food(gdata_features_class_doc2vec)

 10%|████████▎                                                                          | 1/10 [00:03<00:34,  3.88s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.874802371541502 0.0835620822171437
Average and SD of AUP for FM2: 0.8326744802213237 0.09089243739469068
Average and SD of f1 Socre FM2: 0.8699810756519089 0.09416671695220363
------------------------------------------------------------------------------------------------------


 20%|████████████████▌                                                                  | 2/10 [00:07<00:31,  3.94s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8898221343873518 0.07561354957943221
Average and SD of AUP for FM2: 0.8514715420078126 0.08382047481516611
Average and SD of f1 Socre FM2: 0.8859240805853348 0.08382159041191595
------------------------------------------------------------------------------------------------------


 30%|████████████████████████▉                                                          | 3/10 [00:12<00:28,  4.11s/it]

...

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8876482213438736 0.07844387364823571
Average and SD of AUP for FM2: 0.8475500542659588 0.08647883873292693
Average and SD of f1 Socre FM2: 0.8894766144021463 0.07535215943963683
------------------------------------------------------------------------------------------------------


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:36<00:04,  4.08s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8879446640316205 0.06849358747999096
Average and SD of AUP for FM2: 0.8525949380166271 0.0806133971409499
Average and SD of f1 Socre FM2: 0.8843405962981937 0.07452445585767183
------------------------------------------------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:41<00:00,  4.12s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.9100790513833992 0.06568433866190193
Average and SD of AUP for FM2: 0.8774892768262333 0.07945816493595767
Average and SD of f1 Socre FM2: 0.9074742446109552 0.07338395184255064
------------------------------------------------------------------------------------------------------


In [286]:
## Garlic normalized features
# get_cross_validation_seen_food(gdata_features_class_normalized)

------------------------------------------------------------------------------------------------------
Average AUC for FM1 and FM2:  0.7492094861660079 0.78300395256917
Average AUP for FM1 and FM2: 0.7027265131662365 0.7244753841499598
Average f1 Socre FM1 and FM2: 0.7192784453830965 0.7692480587878655
------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
Average AUC for FM1 and FM2:  0.733300395256917 0.7335968379446641
Average AUP for FM1 and FM2: 0.6857352430063061 0.6762415783450266
Average f1 Socre FM1 and FM2: 0.6996408308595237 0.7099844065918886
------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
Average AUC for FM1 and FM2:  0.7237154150197628 0.7533596837944664
Average AUP fo

In [41]:
## Cocoa
get_cross_validation_seen_food(cdata_features_class_doc2vec)

 10%|████████▎                                                                          | 1/10 [00:04<00:37,  4.16s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8706521739130435 0.1166347343553328
Average and SD of AUP for FM2: 0.8391125423896006 0.1297662937815742
Average and SD of f1 Socre FM2: 0.8566773831736237 0.13750291098444886
------------------------------------------------------------------------------------------------------


 20%|████████████████▌                                                                  | 2/10 [00:08<00:33,  4.18s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.9006340579710145 0.09489196320139152
Average and SD of AUP for FM2: 0.8699621539508219 0.11048454277898437
Average and SD of f1 Socre FM2: 0.8935610736040346 0.10740212424824414
------------------------------------------------------------------------------------------------------


 30%|████████████████████████▉                                                          | 3/10 [00:12<00:29,  4.20s/it]

...

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8923007246376812 0.1019675313756365
Average and SD of AUP for FM2: 0.8643747330066291 0.12112985795086745
Average and SD of f1 Socre FM2: 0.8826258113001298 0.11590691182567871
------------------------------------------------------------------------------------------------------


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:41<00:04,  4.62s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8795289855072463 0.10400901744024262
Average and SD of AUP for FM2: 0.8489266146996562 0.11913756442387162
Average and SD of f1 Socre FM2: 0.8670700228511778 0.12483847792457435
------------------------------------------------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:46<00:00,  4.63s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8775362318840579 0.11717934097830941
Average and SD of AUP for FM2: 0.8480074014566688 0.134172134188881
Average and SD of f1 Socre FM2: 0.8658072433554738 0.1327356594806497
------------------------------------------------------------------------------------------------------


In [42]:
## Basil
get_cross_validation_seen_food(bdata_features_class_doc2vec)

 10%|████████▎                                                                          | 1/10 [00:02<00:22,  2.47s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.7266666666666667 0.18336363385966759
Average and SD of AUP for FM2: 0.7144564694564695 0.17444797417599517
Average and SD of f1 Socre FM2: 0.7529331779331779 0.15212567746109126
------------------------------------------------------------------------------------------------------


 20%|████████████████▌                                                                  | 2/10 [00:04<00:19,  2.48s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.7333333333333334 0.12018504251546633
Average and SD of AUP for FM2: 0.6960804473304474 0.12103799208027892
Average and SD of f1 Socre FM2: 0.7515873015873016 0.10428610312977586
------------------------------------------------------------------------------------------------------


 30%|████████████████████████▉                                                          | 3/10 [00:07<00:17,  2.49s/it]

...

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.7966666666666666 0.08938058451861286
Average and SD of AUP for FM2: 0.7686075036075036 0.092892720266533
Average and SD of f1 Socre FM2: 0.7848562548562548 0.08522875585610758
------------------------------------------------------------------------------------------------------


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:23<00:02,  2.67s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.7250000000000001 0.10547511554864494
Average and SD of AUP for FM2: 0.6943777056277056 0.11449950943567495
Average and SD of f1 Socre FM2: 0.7395909645909645 0.08858135075459839
------------------------------------------------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:26<00:00,  2.62s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.7983333333333333 0.10657548185832111
Average and SD of AUP for FM2: 0.7592633477633478 0.1205216569319803
Average and SD of f1 Socre FM2: 0.8107747807747808 0.08730393895510094
------------------------------------------------------------------------------------------------------


In [43]:
## Apple
get_cross_validation_seen_food(adata_features_class_doc2vec)

 10%|████████▎                                                                          | 1/10 [00:23<03:30, 23.41s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8766701680672269 0.0867356866695072
Average and SD of AUP for FM2: 0.8377421988543059 0.09269476823123957
Average and SD of f1 Socre FM2: 0.8687915748040658 0.09864612515438827
------------------------------------------------------------------------------------------------------


 20%|████████████████▌                                                                  | 2/10 [00:45<03:04, 23.09s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.874562324929972 0.07974707891823585
Average and SD of AUP for FM2: 0.8356363391869955 0.0869702404054504
Average and SD of f1 Socre FM2: 0.8678436903452009 0.08977274023497224
------------------------------------------------------------------------------------------------------


 30%|████████████████████████▉                                                          | 3/10 [01:08<02:40, 22.87s/it]

...

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8683088235294117 0.09314660876609085
Average and SD of AUP for FM2: 0.8301402495978785 0.10186936868717034
Average and SD of f1 Socre FM2: 0.8598570162694275 0.10442627460452254
------------------------------------------------------------------------------------------------------


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [03:24<00:22, 22.90s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8775070028011205 0.07568911626675413
Average and SD of AUP for FM2: 0.8387309261489969 0.08094278363950143
Average and SD of f1 Socre FM2: 0.870623325877277 0.08560568499484841
------------------------------------------------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:47<00:00, 22.71s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8842086834733894 0.07905083283870278
Average and SD of AUP for FM2: 0.8470185585905687 0.08688250748217086
Average and SD of f1 Socre FM2: 0.8780513053370393 0.08834323051785145
------------------------------------------------------------------------------------------------------


In [44]:
## Milk
get_cross_validation_seen_food(mdata_features_class_doc2vec)

 10%|████████▎                                                                          | 1/10 [00:02<00:26,  2.98s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8150641025641026 0.04649203907732369
Average and SD of AUP for FM2: 0.7562978628125687 0.0508574606272778
Average and SD of f1 Socre FM2: 0.8193670869083163 0.051346252639820095
------------------------------------------------------------------------------------------------------


 20%|████████████████▌                                                                  | 2/10 [00:06<00:24,  3.00s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.7878205128205128 0.07113363076195423
Average and SD of AUP for FM2: 0.7274456401232419 0.07586545547999103
Average and SD of f1 Socre FM2: 0.7899084249084249 0.07841148752643728
------------------------------------------------------------------------------------------------------


 30%|████████████████████████▉                                                          | 3/10 [00:09<00:21,  3.02s/it]

...

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.7987179487179488 0.04599777743823145
Average and SD of AUP for FM2: 0.7439120532559447 0.04818332502199766
Average and SD of f1 Socre FM2: 0.7964892144892145 0.06141488131417044
------------------------------------------------------------------------------------------------------


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:28<00:03,  3.20s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.7878205128205128 0.06923076923076929
Average and SD of AUP for FM2: 0.7289951249730662 0.07255825387707436
Average and SD of f1 Socre FM2: 0.7906178266178266 0.07339820335705441
------------------------------------------------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:31<00:00,  3.14s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.7871794871794873 0.06509172711532564
Average and SD of AUP for FM2: 0.72842092564717 0.07257263650784145
Average and SD of f1 Socre FM2: 0.7925400764110441 0.06994872168372786
------------------------------------------------------------------------------------------------------


In [45]:
# All
all_foods_doc2vec = pd.concat([gdata_features_class_doc2vec,cdata_features_class_doc2vec,bdata_features_class_doc2vec,adata_features_class_doc2vec,mdata_features_class_doc2vec])
get_cross_validation_seen_food(all_foods_doc2vec)

 10%|████████▎                                                                          | 1/10 [00:30<04:38, 30.99s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8211223400916452 0.09951555453649442
Average and SD of AUP for FM2: 0.7743326531233575 0.1062165125348052
Average and SD of f1 Socre FM2: 0.8124916750042496 0.11310118297278489
------------------------------------------------------------------------------------------------------


 20%|████████████████▌                                                                  | 2/10 [01:06<04:19, 32.44s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8258615278388048 0.09701427666464203
Average and SD of AUP for FM2: 0.7796094141947898 0.10390510061102784
Average and SD of f1 Socre FM2: 0.8161617077551503 0.11117191179512452
------------------------------------------------------------------------------------------------------


 30%|████████████████████████▉                                                          | 3/10 [01:43<03:56, 33.82s/it]

...

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8199893289812316 0.09812171497814434
Average and SD of AUP for FM2: 0.7713244770958276 0.1023031737034177
Average and SD of f1 Socre FM2: 0.8098990177786927 0.11559057440488185
------------------------------------------------------------------------------------------------------


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [05:11<00:34, 34.52s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.831176636745967 0.09303308923268144
Average and SD of AUP for FM2: 0.7833366533892022 0.09891814731716037
Average and SD of f1 Socre FM2: 0.8231267320338997 0.10606001974227407
------------------------------------------------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [05:42<00:00, 34.29s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8247551942753123 0.1043852365283305
Average and SD of AUP for FM2: 0.7782744649060567 0.11010852838173381
Average and SD of f1 Socre FM2: 0.8135464974757284 0.1224972517698028
------------------------------------------------------------------------------------------------------


# Performance on unseen food

In [46]:
def xgboost_model_unseen_food(x_other,y_other,x_food,y_food,kfold,n_splits=10):

    cross_val_model_fm1 = RandomForestClassifier(max_depth=80,random_state=0)
    cross_val_model_fm2 = RandomForestClassifier(max_depth=80,random_state=0)
    
    kfold = StratifiedKFold(n_splits)
    
    auc_fm1 = []
    aup_fm1 = []
    f1_score_list_fm1 = []
    
    auc_fm2 = []
    aup_fm2 = []
    f1_score_list_fm2 = []
    
    for i,(test_food,train_food) in enumerate(kfold.split(x_food,y_food)): # test set is kept larger here 
        
        x_other_old = x_other.drop(columns=['chem_ent_ratio','chem_term_count','bigram_score'])
        x_food_old = x_food.drop(columns=['chem_ent_ratio','chem_term_count','bigram_score'])
        
        cross_val_model_fm2.fit(pd.concat([x_other,x_food.loc[test_food]]),pd.concat([y_other,y_food.loc[test_food]]))
        cross_val_model_fm1.fit(pd.concat([x_other_old,x_food_old.loc[test_food]]),pd.concat([y_other,y_food.loc[test_food]]))
        
        #viz_fm1 = plot_roc_curve(cross_val_model_fm1, x_old.loc[test], y.loc[test],
        #                 name='ROC fold {}'.format(i),
        #                 alpha=0.3, lw=1)
        #viz_fm2 = plot_roc_curve(cross_val_model_fm2, x.loc[test], y.loc[test],
        #                 name='ROC fold {}'.format(i),
        #                 alpha=0.3, lw=1)
        
        y_predicted_fm1 = cross_val_model_fm1.predict(x_food_old.loc[train_food])
        y_predicted_fm2 = cross_val_model_fm2.predict(x_food.loc[train_food])
        
        auc_fm1.append(roc_auc_score(np.array(y_food.loc[train_food].tolist()),y_predicted_fm1))
        aup_fm1.append(average_precision_score(np.array(y_food.loc[train_food].tolist()),y_predicted_fm1))
        f1_score_list_fm1.append(f1_score(np.array(y_food.loc[train_food].tolist()),y_predicted_fm1))
        
        auc_fm2.append(roc_auc_score(np.array(y_food.loc[train_food].tolist()),y_predicted_fm2))
        aup_fm2.append(average_precision_score(np.array(y_food.loc[train_food].tolist()),y_predicted_fm2))
        f1_score_list_fm2.append(f1_score(np.array(y_food.loc[train_food].tolist()),y_predicted_fm2))
        # plot_confusion_matrix(cross_val_model,x.loc[test],y.loc[test])
        
    print('------------------------------------------------------------------------------------------------------')
    #print('Average AUC for FM1 and FM2: ', mean(auc_fm1), mean(auc_fm2))
    #print('Average AUP for FM1 and FM2:', mean(aup_fm1), mean(aup_fm2))
    #print('Average f1 Socre FM1 and FM2:', mean(f1_score_list_fm1), mean(f1_score_list_fm2))
    print('Average and SD of AUC for FM2: ', mean(auc_fm2), np.std(auc_fm2))
    print('Average and SD of AUP for FM2:', mean(aup_fm2), np.std(aup_fm2))
    print('Average and SD of f1 Socre FM2:', mean(f1_score_list_fm2), np.std(f1_score_list_fm2))
    print('------------------------------------------------------------------------------------------------------')

    return mean(auc_fm2), mean(aup_fm2), mean(f1_score_list_fm2)

In [47]:
def get_cross_validation_unseen_food(other_data_features_class,fdata_features_class):

    repeat = 10

    for _ in tqdm(range(repeat)):
        oversample = SMOTE()
        y_other = other_data_features_class.copy()['class']
        X_other = other_data_features_class.copy().drop('class', axis = 1)
        X_other_smote, y_other_smote = oversample.fit_resample(X_other, y_other)
        y_food = fdata_features_class.copy()['class']
        X_food = fdata_features_class.copy().drop('class', axis = 1)
        X_food_smote, y_food_smote = oversample.fit_resample(X_food, y_food)
        kfold = StratifiedKFold(n_splits=10)
        auc_fm2, aup_fm2, f1_fm2 = xgboost_model_unseen_food(X_other_smote, y_other_smote,X_food_smote, y_food_smote,kfold,n_splits=10)
        
    return 

In [48]:
## Garlic
other_foods = pd.concat([cdata_features_class_doc2vec,bdata_features_class_doc2vec,adata_features_class_doc2vec,mdata_features_class_doc2vec])
get_cross_validation_unseen_food(other_foods,gdata_features_class_doc2vec)

 10%|████████▎                                                                          | 1/10 [00:39<05:59, 40.00s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8703557312252964 0.13856334301726794
Average and SD of AUP for FM2: 0.8425675991558231 0.1446334020391158
Average and SD of f1 Socre FM2: 0.8652633890391327 0.14568769000313242
------------------------------------------------------------------------------------------------------


 20%|████████████████▌                                                                  | 2/10 [01:19<05:19, 39.99s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8815217391304349 0.11197221928053912
Average and SD of AUP for FM2: 0.8491034683366699 0.11668853521321335
Average and SD of f1 Socre FM2: 0.878458458908731 0.11645857163485773
------------------------------------------------------------------------------------------------------


 30%|████████████████████████▉                                                          | 3/10 [01:59<04:39, 39.87s/it]

...

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8839920948616601 0.11870645115986735
Average and SD of AUP for FM2: 0.8579687764181578 0.13216096933209712
Average and SD of f1 Socre FM2: 0.8764993920807874 0.12900024488887304
------------------------------------------------------------------------------------------------------


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [05:49<00:37, 37.91s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8726284584980237 0.12765823241308188
Average and SD of AUP for FM2: 0.8474925816157354 0.13615742928778285
Average and SD of f1 Socre FM2: 0.8674882894253285 0.13191054283315526
------------------------------------------------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [06:25<00:00, 38.51s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8771739130434782 0.1264953751180666
Average and SD of AUP for FM2: 0.8517132396358005 0.134181833999699
Average and SD of f1 Socre FM2: 0.8728220162042205 0.12986143152450455
------------------------------------------------------------------------------------------------------


In [295]:
## Garlic normalized features
#other_foods_normalized = pd.concat([cdata_features_class_normalized,bdata_features_class_normalized,adata_features_class_normalized,mdata_features_class_normalized])
#get_cross_validation_unseen_food(other_foods_normalized,gdata_features_class_normalized)

------------------------------------------------------------------------------------------------------
Average AUC for FM1 and FM2:  0.6126520100502513 0.5979057788944724
Average AUP for FM1 and FM2: 0.568960919016526 0.5584431905318619
Average f1 Socre FM1 and FM2: 0.6224184595434816 0.6350375051014985
------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
Average AUC for FM1 and FM2:  0.6124045226130653 0.607925879396985
Average AUP for FM1 and FM2: 0.5692603356353355 0.563863697215411
Average f1 Socre FM1 and FM2: 0.6244422243416897 0.6489699881775981
------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
Average AUC for FM1 and FM2:  0.5838831658291457 0.5976633165829146
Average AUP fo

In [49]:
## Cocoa
other_foods = pd.concat([gdata_features_class_doc2vec,bdata_features_class_doc2vec,adata_features_class_doc2vec,mdata_features_class_doc2vec])
get_cross_validation_unseen_food(other_foods,cdata_features_class_doc2vec)

 10%|████████▎                                                                          | 1/10 [00:34<05:10, 34.45s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8338768115942029 0.10772528817629318
Average and SD of AUP for FM2: 0.7801526892882163 0.1141128855526579
Average and SD of f1 Socre FM2: 0.8306797824658932 0.12109026631653655
------------------------------------------------------------------------------------------------------


 20%|████████████████▌                                                                  | 2/10 [01:08<04:35, 34.39s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8685688405797102 0.07801689906082801
Average and SD of AUP for FM2: 0.8167884658156417 0.09500061977352381
Average and SD of f1 Socre FM2: 0.8715850097031858 0.08036471851641241
------------------------------------------------------------------------------------------------------


 30%|████████████████████████▉                                                          | 3/10 [01:43<04:01, 34.47s/it]

...

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8316123188405797 0.10175488550385453
Average and SD of AUP for FM2: 0.7758367994256061 0.10749130262882546
Average and SD of f1 Socre FM2: 0.8312025540221493 0.11161242600028931
------------------------------------------------------------------------------------------------------


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [05:12<00:34, 34.86s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8425724637681159 0.08600093906864584
Average and SD of AUP for FM2: 0.7845885971433455 0.09353301504893353
Average and SD of f1 Socre FM2: 0.8467018307880566 0.08954744677264352
------------------------------------------------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [05:47<00:00, 34.78s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8380434782608696 0.09573534071092868
Average and SD of AUP for FM2: 0.7816462730647532 0.10112259385704116
Average and SD of f1 Socre FM2: 0.8387883164140774 0.1051169247957342
------------------------------------------------------------------------------------------------------


In [50]:
## Basil
other_foods = pd.concat([gdata_features_class_doc2vec,cdata_features_class_doc2vec,adata_features_class_doc2vec,mdata_features_class_doc2vec])
get_cross_validation_unseen_food(other_foods,bdata_features_class_doc2vec)

 10%|████████▎                                                                          | 1/10 [00:37<05:38, 37.65s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.4816666666666667 0.11936172846529251
Average and SD of AUP for FM2: 0.5307676767676768 0.08351559132575402
Average and SD of f1 Socre FM2: 0.24293650793650795 0.1471730975243799
------------------------------------------------------------------------------------------------------


 20%|████████████████▌                                                                  | 2/10 [01:16<05:03, 37.96s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.49 0.09345230512584124
Average and SD of AUP for FM2: 0.5201010101010101 0.05644232586784951
Average and SD of f1 Socre FM2: 0.20515873015873015 0.1445779297741456
------------------------------------------------------------------------------------------------------


 30%|████████████████████████▉                                                          | 3/10 [01:51<04:20, 37.15s/it]

...

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.495 0.10221165404307973
Average and SD of AUP for FM2: 0.531489898989899 0.06605493461738814
Average and SD of f1 Socre FM2: 0.24857142857142858 0.15387650678347264
------------------------------------------------------------------------------------------------------


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [05:30<00:36, 36.65s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.5166666666666667 0.1376388188137505
Average and SD of AUP for FM2: 0.5568434343434343 0.09554872498383284
Average and SD of f1 Socre FM2: 0.2715873015873016 0.20327310360332002
------------------------------------------------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [06:06<00:00, 36.66s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.5433333333333333 0.10780641085864151
Average and SD of AUP for FM2: 0.5654545454545454 0.0924138096446602
Average and SD of f1 Socre FM2: 0.2119047619047619 0.22219529315293723
------------------------------------------------------------------------------------------------------


In [51]:
## Apple
other_foods = pd.concat([gdata_features_class_doc2vec,cdata_features_class_doc2vec,bdata_features_class_doc2vec,mdata_features_class_doc2vec])
get_cross_validation_unseen_food(other_foods,adata_features_class_doc2vec)

 10%|████████▎                                                                          | 1/10 [00:33<05:03, 33.78s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8716316526610645 0.08332324751243583
Average and SD of AUP for FM2: 0.8290571366115325 0.08725293743708352
Average and SD of f1 Socre FM2: 0.8647800292345609 0.09466512687981132
------------------------------------------------------------------------------------------------------


 20%|████████████████▌                                                                  | 2/10 [01:07<04:30, 33.76s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8825490196078432 0.08361912865604601
Average and SD of AUP for FM2: 0.8435015369493066 0.08940250592556002
Average and SD of f1 Socre FM2: 0.8763434822084755 0.09351548876919932
------------------------------------------------------------------------------------------------------


 30%|████████████████████████▉                                                          | 3/10 [01:40<03:54, 33.54s/it]

...

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8737464985994398 0.07963875621467287
Average and SD of AUP for FM2: 0.8341202608126007 0.08534630310767867
Average and SD of f1 Socre FM2: 0.8667724187349178 0.09008909052160964
------------------------------------------------------------------------------------------------------


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [05:05<00:34, 34.15s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8733333333333333 0.08454620188697526
Average and SD of AUP for FM2: 0.8335548532869065 0.08937300567801269
Average and SD of f1 Socre FM2: 0.865867266754454 0.09538624862956252
------------------------------------------------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [05:38<00:00, 33.81s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.8758193277310925 0.08175835567371313
Average and SD of AUP for FM2: 0.8362425081454596 0.08799810392459599
Average and SD of f1 Socre FM2: 0.8687339461134617 0.0928397213732703
------------------------------------------------------------------------------------------------------


In [52]:
## Milk
other_foods = pd.concat([gdata_features_class_doc2vec,cdata_features_class_doc2vec,bdata_features_class_doc2vec,adata_features_class_doc2vec])
get_cross_validation_unseen_food(other_foods,mdata_features_class_doc2vec)

 10%|████████▎                                                                          | 1/10 [00:37<05:37, 37.52s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.6951923076923077 0.06720764848188167
Average and SD of AUP for FM2: 0.6666262626262627 0.06613956006254185
Average and SD of f1 Socre FM2: 0.6070869405941421 0.11803460003619005
------------------------------------------------------------------------------------------------------


 20%|████████████████▌                                                                  | 2/10 [01:12<04:54, 36.76s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.6868589743589744 0.09091553922650854
Average and SD of AUP for FM2: 0.6669095071595071 0.10292411411638965
Average and SD of f1 Socre FM2: 0.5973082327029695 0.12595640120935414
------------------------------------------------------------------------------------------------------


 30%|████████████████████████▉                                                          | 3/10 [01:48<04:16, 36.61s/it]

...

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.6669871794871796 0.09096524269214455
Average and SD of AUP for FM2: 0.6411207264957265 0.0943697744096545
Average and SD of f1 Socre FM2: 0.5704364914261939 0.1411932296944331
------------------------------------------------------------------------------------------------------


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [05:29<00:36, 36.21s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.6753205128205129 0.0785230245508627
Average and SD of AUP for FM2: 0.6462296592296592 0.08103460651629943
Average and SD of f1 Socre FM2: 0.60339447102605 0.09608318640702201
------------------------------------------------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [06:07<00:00, 36.77s/it]

------------------------------------------------------------------------------------------------------
Average and SD of AUC for FM2:  0.691025641025641 0.08534552889923451
Average and SD of AUP for FM2: 0.6630000277500278 0.09304324116458235
Average and SD of f1 Socre FM2: 0.6206010530278265 0.121152508362762
------------------------------------------------------------------------------------------------------


# UMAP visualization 

In [53]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as clr
from collections import Counter

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import umap

from sklearn.cluster import KMeans, DBSCAN

import numpy as np
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, plot_mpl
init_notebook_mode(connected=True)

In [54]:
gtrain = pd.read_csv("data/garlic_scoring.csv", encoding='latin1')[['PMID', 'abstract', 'paper', 'mesh_terms', 'qual_terms', 'is_useful']]
gtrain['food'] = 'garlic'
ctrain = pd.read_csv("data/cocoa_scoring.csv", encoding='latin1')[['PMID', 'abstract', 'paper', 'mesh_terms', 'qual_terms', 'is_useful']]
ctrain['food'] = 'cocoa'
gtrain = gtrain[gtrain['is_useful'].notnull()]
ctrain = ctrain[ctrain['is_useful'].notnull()]

In [55]:
gdata_features_class_doc2vec['class'] = gtrain['is_useful'].tolist()
cdata_features_class_doc2vec['class'] = ctrain['is_useful'].tolist()

In [56]:
all_foods_doc2vec = pd.concat([gdata_features_class_doc2vec,cdata_features_class_doc2vec,bdata_features_class_doc2vec,adata_features_class_doc2vec,mdata_features_class_doc2vec])

food_name_list = []
for _ in range(len(gdata_features_class_doc2vec)):
    food_name_list.append('garlic')
for _ in range(len(cdata_features_class_doc2vec)):
    food_name_list.append('cocoa')
for _ in range(len(bdata_features_class_doc2vec)):
    food_name_list.append('basil')
for _ in range(len(adata_features_class_doc2vec)):
    food_name_list.append('apple')
for _ in range(len(mdata_features_class_doc2vec)):
    food_name_list.append('human milk')

all_foods_doc2vec['food_name'] = food_name_list

In [57]:
all_foods_doc2vec_features = all_foods_doc2vec.drop(columns=['class','food_name'])
all_foods_doc2vec['concatenated_embedding'] = all_foods_doc2vec_features.values.tolist()

In [58]:
embeddings = all_foods_doc2vec['concatenated_embedding'].tolist()

In [59]:
reducer = umap.UMAP(n_components=3)
umap_fit = reducer.fit_transform(np.array(embeddings))

C:\Users\Ayan\anaconda3\lib\site-packages\umap\spectral.py:229: UserWarning:

Embedding a total of 2 separate connected components using meta-embedding (experimental)



In [60]:
viz_data_full = pd.DataFrame(umap_fit, columns = ['umap_0', 'umap_1', 'umap_2'])
viz_data_full['is_useful'] = all_foods_doc2vec['class'].tolist()
viz_data_full['food'] = all_foods_doc2vec['food_name'].tolist()
viz_data_full

,umap_0,umap_1,umap_2,is_useful,food
0,13.576961,2.384406,3.442932,1.0,garlic
1,13.777128,3.247021,4.353684,0.0,garlic
2,18.548460,2.746119,3.322711,0.0,garlic
3,15.005014,1.965498,3.574461,0.0,garlic
4,16.695442,2.086138,3.052322,2.0,garlic
...,...,...,...,...,...
2599,14.126314,3.582968,3.064538,1.0,human milk
2600,18.857845,3.333478,2.200412,1.0,human milk
2601,18.647823,3.648624,1.830497,1.0,human milk
2602,18.358011,3.815198,1.590334,0.0,human milk


In [61]:
# Garlic
viz_data = viz_data_full[viz_data_full['food'] == 'garlic']

In [62]:
def fill_na_w_list(v):
    if not isinstance(v, list):
        return []
    else:
        return v


def make_label(row, name='chem'):
    return #'<br>Abstract: ' + row['is_useful'] #\
        #+ '<br>InChiKey: ' + row['InChiKey'] \
        #+ '<br>cid: ' + row['cid']

traces = []
# class_col = 'chebi_class'
# class_col = 'Super Class'
class_col = 'is_useful'
# class_col = 'cluster'
color_list = ['#fa9fb5','#377eb8','#1b9e77']

for i, c in enumerate(set(viz_data[class_col].dropna().drop_duplicates().tolist()) - set(['NA', '-1'])):
    print(c)
    traces.append( go.Scatter3d(
        x=viz_data[viz_data[class_col] == c].umap_0.tolist(),
        y=viz_data[viz_data[class_col] == c].umap_1.tolist(),
        z=viz_data[viz_data[class_col] == c].umap_2.tolist(),
        mode='markers',
        marker=dict(
            opacity=0.7,
            size=3,
            color=  color_list[i] # sns.color_palette("hsv", 17,desat=.7).as_hex()[i]
        ),
        name=c,
        text=viz_data[viz_data[class_col] == c].apply(make_label, axis=1),
        hoverinfo='text'
#         customdata=viz_data[viz_data['Super Class'] == c].diseases.tolist()
    ))


#v2 = viz_data[viz_data[class_col].isnull()].copy()
#trace2 = go.Scatter3d(
#    x=v2.umap_0.tolist(),
#    y=v2.umap_1.tolist(),
#    z=v2.umap_2.tolist(),
#    mode='markers',
#    marker=dict(
#        opacity=0.05,
#        size=3,
#        color='lightgray'
#    ),
#    name='No Class Labels',
#    text=v2.apply(make_label, axis=1),
#    hoverinfo='text'
#)

#traces = [trace2] + traces

layout = go.Layout(
    title='FoodMine Space | Garlic'
)

fig = go.Figure(data=traces, layout=layout)

fn = 'FoodMine_Space_Garlic_2'
# fig.write_image(f"figs/{fn}.svg")
plot(fig, filename = f'{fn}.html', auto_open=False)

0.0
1.0
2.0


'FoodMine_Space_Garlic_2.html'

In [63]:
# Cocoa
viz_data = viz_data_full[viz_data_full['food'] == 'cocoa']

In [64]:
def fill_na_w_list(v):
    if not isinstance(v, list):
        return []
    else:
        return v


def make_label(row, name='chem'):
    return #'<br>Abstract: ' + row['is_useful'] #\
        #+ '<br>InChiKey: ' + row['InChiKey'] \
        #+ '<br>cid: ' + row['cid']

traces = []
# class_col = 'chebi_class'
# class_col = 'Super Class'
class_col = 'is_useful'
# class_col = 'cluster'
color_list = ['#fa9fb5','#377eb8','#1b9e77']

for i, c in enumerate(set(viz_data[class_col].dropna().drop_duplicates().tolist()) - set(['NA', '-1'])):
    print(c)
    traces.append( go.Scatter3d(
        x=viz_data[viz_data[class_col] == c].umap_0.tolist(),
        y=viz_data[viz_data[class_col] == c].umap_1.tolist(),
        z=viz_data[viz_data[class_col] == c].umap_2.tolist(),
        mode='markers',
        marker=dict(
            opacity=0.7,
            size=3,
            color=  color_list[i] # sns.color_palette("hsv", 17,desat=.7).as_hex()[i]
        ),
        name=c,
        text=viz_data[viz_data[class_col] == c].apply(make_label, axis=1),
        hoverinfo='text'
#         customdata=viz_data[viz_data['Super Class'] == c].diseases.tolist()
    ))


#v2 = viz_data[viz_data[class_col].isnull()].copy()
#trace2 = go.Scatter3d(
#    x=v2.umap_0.tolist(),
#    y=v2.umap_1.tolist(),
#    z=v2.umap_2.tolist(),
#    mode='markers',
#    marker=dict(
#        opacity=0.05,
#        size=3,
#        color='lightgray'
#    ),
#    name='No Class Labels',
#    text=v2.apply(make_label, axis=1),
#    hoverinfo='text'
#)

#traces = [trace2] + traces

layout = go.Layout(
    title='FoodMine Space | Cocoa'
)

fig = go.Figure(data=traces, layout=layout)

fn = 'FoodMine_Space_Cocoa_2'
# fig.write_image(f"figs/{fn}.svg")
plot(fig, filename = f'{fn}.html', auto_open=False)

0.0
1.0
2.0


'FoodMine_Space_Cocoa_2.html'